In [5]:
#imports
import json
import os
import warnings
from datetime import date, datetime, timedelta

import pandas as pd
import requests
from sqlalchemy import create_engine

warnings.filterwarnings("ignore")

MORNING_LEAD_LIST_ID = 214864
MORNING_BROADCAST_ID = 82319
EVENING_LEAD_LIST_ID = 214867
EVENING_BROADCAST_ID = 82321

## auth token 
# from login import auth_token

#db information
user1 = "SFPL_Connect"
password = "$%n5bF33%X"
db = "Sonata_Connect"
server = "172.17.130.216"
engine = create_engine(f"mssql+pyodbc://{user1}:{password}@{server}/{db}?driver=ODBC+Driver+17+for+SQL+Server")

def get_authorization():
    """ Get auth token """
    username = os.environ['PAYMEE_SONATA_TATA_USERNAME']
    password = os.environ['PAYMEE_SONATA_TATA_PASSWORD']
    print(username,password)
    url = "https://api-smartflo.tatateleservices.com/v1/auth/login"

    payload = {
        "email": username,
        "password": password
    }
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json"
    }

    response = requests.post(url, json=payload, headers=headers)
    if response.status_code == 200:
        return response.json()['access_token']
    else :
        return None
    
auth_token = get_authorization()


TACN2479 SonataFin123@@


In [6]:
pd.set_option('display.max_columns', 500)

In [8]:

today_date = str(date.today())
today_date = datetime.strptime(today_date, '%Y-%m-%d').strftime('%Y-%m-%d')
sql_query = f""" SET NOCOUNT ON;EXEC [{db}].[dbo].[SP_UserHierarchy_Dynamic_07Jan23] @userid = 1;SET NOCOUNT OFF """
hierarchy_df=pd.read_sql_query(sql_query,engine)

OperationalError: (pyodbc.OperationalError) ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]Named Pipes Provider: Could not open a connection to SQL Server [53].  (53) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]A network-related or instance-specific error has occurred while establishing a connection to SQL Server. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (53)')
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
bro_df = hierarchy_df[hierarchy_df['RoleId'] == 55][['UserID', 'UserName']]

In [ ]:
bro_df.rename(columns={'UserID' : 'BRO_UserID', 'UserName':'BRO_UserName'}, inplace=True)
bro_df

,BRO_UserID,BRO_UserName
255,4829,ANKIT
369,3353,RAHUL KUMAR
371,25715,DHEERAJ KUMAR
372,27274,Saurabh Kushwaha
373,31073,NEERAJ KUMAR
...,...,...
3009,31751,SAKSHI CHOUDHARY
3011,27196,Mithilesh Kumar Paswan
3013,31803,SHANU KUMAR
3014,31817,MODASSHIR RAZA


In [ ]:
sql_query = f"""SET NOCOUNT ON;EXEC [{db}].[dbo].[SP_UserHierarchy_Dynamic_07Jan23] @userid = 1;SET NOCOUNT OFF"""
df=pd.read_sql_query(sql_query,engine)
df = df.replace(['Prayagraj Division','Lucknow Division'],['Prayagraj', 'Lucknow'])

df = df.replace(['Prayagraj Division','Lucknow Division'],['Prayagraj', 'Lucknow'])
#WE CREATE A NEW USER HIERARCHY TABLE
div_df = df[(df['RoleId']==34)&(df['RoleName']=='DIVISION HEAD')][['U_BUID','UserID','UserName','buname']]
print(len(div_df))
div_df.rename(columns={'U_BUID':'Div_BUID','buname':'DivName','UserID':'Div_UserID','UserName':'Div_UserName'}, inplace=True)
reg_df = df[(df['RoleId']==35)&(df['RoleName']=='REGION HEAD')][['U_BUID','UserID','UserName','buname','ReportingBUId']]
print(len(reg_df))
reg_df.rename(columns={'U_BUID':'Region_BUID','ReportingBUId':'Region_ReportingBUId','buname':'RegionName','UserID':'Region_UserID','UserName':'Region_UserName'}, inplace=True)
# hub_df = df[(df['RoleId']==44)&(df['RoleName']=='Credit Committee')][['U_BUID','UserID','UserName','buname','ReportingBUId']]
hub_df = df[(df['RoleId'].isin([36,44,42]))&(df['RoleName'].isin(['Credit Committee','Credit Officer','HUB HEAD']))][['U_BUID','UserID','UserName','buname','ReportingBUId','RoleId']]
#We will classify HUB,CRC,CRO on the basis of RID
hub_df.rename(columns={'U_BUID':'Hub_BUID','ReportingBUId':'Hub_ReportingBUId','buname':'HubName','UserID':'Hub_UserID','UserName':'Hub_UserName','RoleId':'RID'}, inplace=True)
bm_df = df[(df['RoleId']==13)&(df['RoleName']=='Branch Manager')][['U_BUID','buname','UserID','UserName','ReportingBUId']]
bm_df.rename(columns={'U_BUID':'BM_BUID','ReportingBUId':'BM_ReportingBUId','buname':'BranchName','UserID':'BM_UserID','UserName':'BM_UserName'}, inplace=True)
bro_df = df[(df['RoleId']==55)&(df['RoleName']=='BRO')][['U_BUID','UserID','UserName','buname']]
bro_df.rename(columns={'U_BUID':'BRO_BUID','UserID':'BRO_UserID','UserName':'BRO_UserName','buname':'BRO_buname'}, inplace=True)
M1 = pd.merge(bro_df,bm_df,left_on='BRO_BUID',right_on='BM_BUID',how='left').drop_duplicates(subset='BRO_UserID')
M2 = pd.merge(M1,hub_df,left_on='BM_ReportingBUId',right_on='Hub_BUID',how='left').drop_duplicates(subset='BRO_UserID')
M3 = pd.merge(M2,reg_df,left_on='Hub_ReportingBUId',right_on='Region_BUID',how='left').drop_duplicates(subset='BRO_UserID')
dF = pd.merge(M3,div_df,left_on='Region_ReportingBUId',right_on='Div_BUID',how='left').drop_duplicates(subset='BRO_UserID')

11
45


In [ ]:

cols_to_fill = ['BM_BUID','BM_UserID','BM_ReportingBUId','Hub_BUID','Hub_UserID','Hub_ReportingBUId','RID', 'Region_BUID','Region_UserID','Region_ReportingBUId','Div_BUID','Div_UserID']
dF[cols_to_fill] = dF[cols_to_fill].fillna(0).astype('int', errors='ignore')

In [ ]:
dF

,BRO_BUID,BRO_UserID,BRO_UserName,BRO_buname,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,RID,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
0,591,4829,ANKIT,Badaun,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0,0,0,NaN,NaN
1,2,3353,RAHUL KUMAR,Sirathu,2,Sirathu,4797,PINTU KUMAR YADAV,545,545,0,HUB HEAD NOT POSTED,BHARWARI,2180,36,2180,18044,KAMLESH YADAV,Kaushambi,650,650,9739,LAKSHMAN SINGH,Prayagraj
2,2,25715,DHEERAJ KUMAR,Sirathu,2,Sirathu,4797,PINTU KUMAR YADAV,545,545,0,HUB HEAD NOT POSTED,BHARWARI,2180,36,2180,18044,KAMLESH YADAV,Kaushambi,650,650,9739,LAKSHMAN SINGH,Prayagraj
3,2,27274,Saurabh Kushwaha,Sirathu,2,Sirathu,4797,PINTU KUMAR YADAV,545,545,0,HUB HEAD NOT POSTED,BHARWARI,2180,36,2180,18044,KAMLESH YADAV,Kaushambi,650,650,9739,LAKSHMAN SINGH,Prayagraj
4,2,31073,NEERAJ KUMAR,Sirathu,2,Sirathu,4797,PINTU KUMAR YADAV,545,545,0,HUB HEAD NOT POSTED,BHARWARI,2180,36,2180,18044,KAMLESH YADAV,Kaushambi,650,650,9739,LAKSHMAN SINGH,Prayagraj
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2086,2324,31751,SAKSHI CHOUDHARY,Aadhartal,2324,Aadhartal,26898,Ruby Dahiya,562,562,1175,Uma Pilley,JABALPUR URBAN,1007,36,1007,1174,shama parveen,Jabalpur,652,652,15190,JITENDRA YADAV,Jabalpur
2087,2325,27196,Mithilesh Kumar Paswan,Nirmali,2325,Nirmali,25630,Dharmveer Kumar,2104,2104,3904,RANDHEER KUMAR PASWAN,Madhubani,2198,36,2198,10848,SHYAM KISHOR MISHRA,Darbhanga,653,653,10849,KESHRAJ YADAV,Muzaffarpur
2088,2326,31803,SHANU KUMAR,Chandwa,2326,Chandwa,27375,Rahul Raj,2206,2206,1668,RAJESH KUMAR,Chatra,2201,36,2201,17841,MURARI MAURYA,Hazaribagh RO,2200,2200,22544,Shiv Kumar Singh,Ranchi DO
2089,2326,31817,MODASSHIR RAZA,Chandwa,2326,Chandwa,27375,Rahul Raj,2206,2206,1668,RAJESH KUMAR,Chatra,2201,36,2201,17841,MURARI MAURYA,Hazaribagh RO,2200,2200,22544,Shiv Kumar Singh,Ranchi DO


task count

In [ ]:
today_date = datetime.strptime(str(date.today()), '%Y-%m-%d').strftime('%Y-%m-%d')
sql_query = f""" SET NOCOUNT ON;EXEC [{db}].[dbo].[SP_UserHierarchy_Dynamic_07Jan23] @userid = 1;SET NOCOUNT OFF """
hierarchy_df=pd.read_sql_query(sql_query,engine)
hierarchy_df = hierarchy_df[hierarchy_df['RoleId']==55]
hierarchy_df = hierarchy_df[['UserID','UserName']]
userid_tuple = tuple(hierarchy_df['UserID'].unique().tolist())
sql_query = f""" SELECT CallingNumber,UserID FROM Sonata_Connect.dbo.accounts_calling_number_list
    WHERE UserID in {userid_tuple} """
contact_df = pd.read_sql_query(sql_query,engine)
main_df = pd.merge(hierarchy_df,contact_df,on='UserID',how='right')
# sql_query = f""" SELECT TOP 10 assigned_to,status,deadline_date FROM Sonata_Connect.dbo.accounts_task_details
#     WHERE assigned_to in {userid_tuple} and deadline_date = {today_date} """
sql_query = f""" SELECT assigned_to,status,deadline_date,category FROM Sonata_Connect.dbo.accounts_task_details
    WHERE deadline_date = '{today_date}' and category in ('3','5','7','10')"""

task_df = pd.read_sql_query(sql_query,engine)
task_df = task_df[(task_df['status'] == 'Pending')|(task_df['status'] == 'Re-Schedule')]
pending_task_df = task_df.groupby('assigned_to')['deadline_date'].count().reset_index()
pending_task_df = pending_task_df.rename(columns={'deadline_date':'Pending_Count'})

print("Length of pending taks : ", len(pending_task_df))

collection_task_df = task_df[task_df['category']=='3']
collection_task_df = collection_task_df.groupby('assigned_to')['deadline_date'].count().reset_index()
collection_task_df = collection_task_df.rename(columns={'deadline_date':'collection_count'})

proposal_task_df = task_df[task_df['category']=='5']
proposal_task_df = proposal_task_df.groupby('assigned_to')['deadline_date'].count().reset_index()
proposal_task_df = proposal_task_df.rename(columns={'deadline_date':'proposal_count'})

retention_task_df = task_df[task_df['category']=='7']
retention_task_df = retention_task_df.groupby('assigned_to')['deadline_date'].count().reset_index()
retention_task_df = retention_task_df.rename(columns={'deadline_date':'retention_count'})

wrong_number_task_df = task_df[task_df['category']=='10']
wrong_number_task_df = wrong_number_task_df.groupby('assigned_to')['deadline_date'].count().reset_index()
wrong_number_task_df = wrong_number_task_df.rename(columns={'deadline_date':'wrong_number_count'})

task_df = task_df[(task_df['status'] == 'Completed')]
completed_task_df = task_df.groupby('assigned_to')['deadline_date'].count().reset_index()
completed_task_df = completed_task_df.rename(columns={'deadline_date':'Completed_Count'})

main_df = pd.merge(main_df,pending_task_df,left_on='UserID',right_on='assigned_to',how='right')
main_df = pd.merge(main_df,collection_task_df,left_on='UserID',right_on='assigned_to',how='left')
main_df = pd.merge(main_df,proposal_task_df,left_on='UserID',right_on='assigned_to',how='left')
main_df = pd.merge(main_df,retention_task_df,left_on='UserID',right_on='assigned_to',how='left')
main_df = pd.merge(main_df,wrong_number_task_df,left_on='UserID',right_on='assigned_to',how='left')


main_df = main_df.drop(columns=['assigned_to_y', 'assigned_to_x'], axis=1).dropna(subset=['UserID']).fillna(0).reset_index(drop=True)

main_df['collection_count'] = main_df['collection_count'].astype(int, errors='ignore')
main_df['proposal_count'] = main_df['proposal_count'].astype(int, errors='ignore')
main_df['retention_count'] = main_df['retention_count'].astype(int, errors='ignore')
main_df['wrong_number_count'] = main_df['wrong_number_count'].astype(int, errors='ignore')

main_df['UserID'] = main_df['UserID'].astype(int, errors='ignore')
main_df['UserName'] = main_df['UserName'].apply(lambda x : x.title())

Length of pending taks :  650


In [ ]:
main_df = main_df.drop('assigned_to', axis=1)

In [ ]:
main_df.rename(columns={'UserID' : 'BRO_UserID', 'UserName' : 'BRO_UserName'}, inplace=True)
main_df

,BRO_UserID,BRO_UserName,CallingNumber,Pending_Count,collection_count,proposal_count,retention_count,wrong_number_count
0,1709,Ballu,8960154523,1,1,0,0,0
1,1934,Vijay Kumar,7897997362,2,1,0,1,0
2,2114,Vinod Kumar,7897997269,2,1,0,1,0
3,2911,Deepak Singh,8218048477,1,0,0,1,0
4,2949,Dhanpat Singh,7080200380,2,2,0,0,0
...,...,...,...,...,...,...,...,...
590,31793,Manish Kumar,8299339722,2,2,0,0,0
591,31827,Sumit Kumar,8192892088,5,5,0,0,0
592,31836,Mukesh Verma,6261403746,2,0,0,2,0
593,31907,Akhilesh Kumar,8292195969,3,3,0,0,0


In [ ]:
dF[dF['BRO_UserID'] == 31810]

,BRO_BUID,BRO_UserID,BRO_UserName,BRO_buname,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,RID,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
1078,357,31810,PANKAJ KUMAR,Mashrakh,357,Mashrakh,4964,SUMIT KUMAR,605,605,3852,NANDLAL RAM,Chapra,1012,36,1012,24776,RAJNAND MAURYA,Muzaffarpur,653,653,10849,KESHRAJ YADAV,Muzaffarpur


In [ ]:
main_df = pd.merge(left=main_df, right=dF, left_on='BRO_UserID', right_on='BRO_UserID', how='left').drop('BRO_UserName_y', axis=1).rename(columns={'BRO_UserName_x' : 'BRO_UserName'})

In [ ]:
main_df

,BRO_UserID,BRO_UserName,CallingNumber,Pending_Count,collection_count,proposal_count,retention_count,wrong_number_count,BRO_BUID,BRO_buname,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,Hub_UserID,Hub_UserName,HubName,Hub_ReportingBUId,RID,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
0,1709,Ballu,8960154523,1,1,0,0,0,92,Handia,92,Handia,2896,KARUNA SAGAR,2132,2132,3883,RAVINDRA KUMAR,HANDIA,1001,36,1001,7114,Akash Jyoti singh,Prayagraj,650,650,9739,LAKSHMAN SINGH,Prayagraj
1,1934,Vijay Kumar,7897997362,2,1,0,1,0,97,Naini,97,Naini,1069,PRATIMA KUSHAWAHA,2222,2222,7611,Arvind Kumar,Gohaniya,2180,36,2180,18044,KAMLESH YADAV,Kaushambi,650,650,9739,LAKSHMAN SINGH,Prayagraj
2,2114,Vinod Kumar,7897997269,2,1,0,1,0,41,Gohaniya,41,Gohaniya,689,Kalpana,2222,2222,7611,Arvind Kumar,Gohaniya,2180,36,2180,18044,KAMLESH YADAV,Kaushambi,650,650,9739,LAKSHMAN SINGH,Prayagraj
3,2911,Deepak Singh,8218048477,1,0,0,1,0,201,Nagal,201,Nagal,17610,NASEEM,601,601,7050,Rakesh Kumar,Saharanpur,1004,44,1004,18034,GRIJESH CHAUDHARY,Saharanpur,2157,2157,10846,SHASHI BHUSHAN SINGH,Saharanpur
4,2949,Dhanpat Singh,7080200380,2,2,0,0,0,300,Atarra,300,Atarra,2907,RAMESH KUMAR PANDEY,548,548,1359,DIWAKAR SAHU,KARVI,2180,36,2180,18044,KAMLESH YADAV,Kaushambi,650,650,9739,LAKSHMAN SINGH,Prayagraj
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,31793,Manish Kumar,8299339722,2,2,0,0,0,79,Raibareli,79,Raibareli,1884,Vinod Kumar,584,584,3438,Dileep Kumar,Raibarely,1002,44,1002,0,REGION HEAD NOT POSTED,Lucknow,651,651,7535,RAKESH KUMAR CHAUDHARY,Lucknow
591,31827,Sumit Kumar,8192892088,5,5,0,0,0,200,Vilashpur,200,Vilashpur,16428,BRIJPAL,599,599,17668,Mohit Chauhan,Ladwa,2178,36,2178,4708,DIPENDRA CHAUHAN,Karnal,2157,2157,10846,SHASHI BHUSHAN SINGH,Saharanpur
592,31836,Mukesh Verma,6261403746,2,0,0,2,0,2096,Rampur naikin,2096,Rampur naikin,7328,Arvind Kumar Kushvaha,2094,2094,18027,PRABHAT SINGH CHOUHAN,Sidhi,2182,36,2182,18023,Vinayak Maha,Rewa,652,652,15190,JITENDRA YADAV,Jabalpur
593,31907,Akhilesh Kumar,8292195969,3,3,0,0,0,511,Chanpattia,511,Chanpattia,5055,SANOJ KUMAR,2273,2273,24860,Santosh Kumar,Narkatiaganj,1020,36,1020,10850,Parasnath maorya,Motihari,653,653,10849,KESHRAJ YADAV,Muzaffarpur


In [ ]:
main_df.columns

Index(['BRO_UserID', 'BRO_UserName', 'CallingNumber', 'Pending_Count',
       'collection_count', 'proposal_count', 'retention_count',
       'wrong_number_count', 'BRO_BUID', 'BRO_buname', 'BM_BUID', 'BranchName',
       'BM_UserID', 'BM_UserName', 'BM_ReportingBUId', 'Hub_BUID',
       'Hub_UserID', 'Hub_UserName', 'HubName', 'Hub_ReportingBUId', 'RID',
       'Region_BUID', 'Region_UserID', 'Region_UserName', 'RegionName',
       'Region_ReportingBUId', 'Div_BUID', 'Div_UserID', 'Div_UserName',
       'DivName'],
      dtype='object')

In [ ]:
brogroup_df = main_df.groupby('BRO_UserID').agg({
    'BRO_UserName' : 'first',
    'Pending_Count' : 'sum',
    'collection_count' : 'sum',
    'proposal_count' : 'sum',
    'retention_count' : 'sum',
    'wrong_number_count' : 'sum',
}).reset_index().rename(columns={'BRO_UserID' : 'UserID', 'BRO_UserName' : 'UserName' })

userid_tuple = tuple(brogroup_df['UserID'].unique().tolist())
sql_query = f""" SELECT CallingNumber,UserID FROM Sonata_Connect.dbo.accounts_calling_number_list
    WHERE UserID in {userid_tuple} """
contact_df = pd.read_sql_query(sql_query,engine)
brogroup_df = pd.merge(brogroup_df,contact_df, on='UserID' , how='left').dropna()
brogroup_df

,UserID,UserName,Pending_Count,collection_count,proposal_count,retention_count,wrong_number_count,CallingNumber
0,1709,Ballu,1,1,0,0,0,8960154523
1,1934,Vijay Kumar,2,1,0,1,0,7897997362
2,2114,Vinod Kumar,2,1,0,1,0,7897997269
3,2911,Deepak Singh,1,0,0,1,0,8218048477
4,2949,Dhanpat Singh,2,2,0,0,0,7080200380
...,...,...,...,...,...,...,...,...
590,31793,Manish Kumar,2,2,0,0,0,8299339722
591,31827,Sumit Kumar,5,5,0,0,0,8192892088
592,31836,Mukesh Verma,2,0,0,2,0,6261403746
593,31907,Akhilesh Kumar,3,3,0,0,0,8292195969


In [ ]:
bmgroup_df = main_df.groupby('BM_UserID').agg({
    'BM_UserName' : 'first',
    'Pending_Count' : 'sum',
    'collection_count' : 'sum',
    'proposal_count' : 'sum',
    'retention_count' : 'sum',
    'wrong_number_count' : 'sum',
}).reset_index().rename(columns={'BM_UserID' : 'UserID', 'BM_UserName' : 'UserName' })

userid_tuple = tuple(bmgroup_df['UserID'].unique().tolist())
sql_query = f""" SELECT CallingNumber,UserID FROM Sonata_Connect.dbo.accounts_calling_number_list
    WHERE UserID in {userid_tuple} """
contact_df = pd.read_sql_query(sql_query,engine)
bmgroup_df = pd.merge(bmgroup_df,contact_df, on='UserID' , how='left').dropna()
bmgroup_df

,UserID,UserName,Pending_Count,collection_count,proposal_count,retention_count,wrong_number_count,CallingNumber
0,16,Manoj Kumar,4,3,0,1,0,7897997270
1,79,Santosh kumar srivastava,1,1,0,0,0,7388898630
2,109,SANJAY VERMA,4,4,0,0,0,8400333636
3,118,Jaswant Singh Chouhan,6,6,0,0,0,7024118524
4,216,VIKAS KUMAR,4,0,0,4,0,7895240962
...,...,...,...,...,...,...,...,...
297,27156,RAJ KUMAR,3,1,0,2,0,8429524667
298,27395,Digvijay Rahangdale,9,4,0,5,0,7067125595
299,27586,Rakesh Kumar,4,0,0,0,4,9835862483
300,27680,Ram Uday ray,4,4,0,0,0,7488211793


In [ ]:
hubgroup_df = main_df.groupby('Hub_UserID').agg({
    'Hub_UserName' : 'first',
    'Pending_Count' : 'sum',
    'collection_count' : 'sum',
    'proposal_count' : 'sum',
    'retention_count' : 'sum',
    'wrong_number_count' : 'sum',
}).reset_index().rename(columns={'Hub_UserID' : 'UserID', 'Hub_UserName' : 'UserName' })

userid_tuple = tuple(hubgroup_df['UserID'].unique().tolist())
sql_query = f""" SELECT CallingNumber,UserID FROM Sonata_Connect.dbo.accounts_calling_number_list
    WHERE UserID in {userid_tuple} """
contact_df = pd.read_sql_query(sql_query,engine)
hubgroup_df = pd.merge(hubgroup_df,contact_df, on='UserID' , how='left').dropna()
hubgroup_df

,UserID,UserName,Pending_Count,collection_count,proposal_count,retention_count,wrong_number_count,CallingNumber
1,119,Shrikant Patel,14,12,0,2,0,7869501423
2,127,Rang Nath Yadav,1,0,0,1,0,7897996918
3,214,Vimal Singh,11,6,0,5,0,7897997252
4,215,YASH PAL,15,14,0,0,1,7897995028
5,259,Vikash Namdev,13,13,0,0,0,9109135682
...,...,...,...,...,...,...,...,...
92,24795,Saurabh Srivastava,3,3,0,0,0,8400333192
93,24807,Mahipal,2,1,0,0,1,7389940066
94,24849,RITESH KUMAR,10,7,0,3,0,8400333376
95,24860,Santosh Kumar,24,23,0,1,0,8887535023


In [ ]:
regiongroup_df = main_df.groupby('Region_UserID').agg({
    'Region_UserName' : 'first',
    'Pending_Count' : 'sum',
    'collection_count' : 'sum',
    'proposal_count' : 'sum',
    'retention_count' : 'sum',
    'wrong_number_count' : 'sum',
}).reset_index().rename(columns={'Region_UserID' : 'UserID', 'Region_UserName' : 'UserName' })

userid_tuple = tuple(regiongroup_df['UserID'].unique().tolist())
sql_query = f""" SELECT CallingNumber,UserID FROM Sonata_Connect.dbo.accounts_calling_number_list
    WHERE UserID in {userid_tuple} """
contact_df = pd.read_sql_query(sql_query,engine)
regiongroup_df = pd.merge(regiongroup_df,contact_df, on='UserID' , how='left').dropna()
regiongroup_df

,UserID,UserName,Pending_Count,collection_count,proposal_count,retention_count,wrong_number_count,CallingNumber
1,1174,shama parveen,25,24,0,1,0,7869501418
2,1622,Pradeep kachhi,25,19,0,2,4,7389941015
3,1968,sompal,22,17,0,5,0,7080211923
4,4347,SHIV BAHADUR,32,26,0,6,0,8400333575
5,4708,DIPENDRA CHAUHAN,148,142,0,3,3,8400333583
6,4891,Praveen Singh,110,105,0,0,5,8400333160
7,7074,Sudheer,2,2,0,0,0,8400333732
9,7342,Neeraj maurya,22,15,0,5,2,8400333299
10,7664,Vikash Singh,39,28,0,9,2,7521901507
11,10848,SHYAM KISHOR MISHRA,26,26,0,0,0,8400333720


In [ ]:
divgroup_df = main_df.groupby('Div_UserID').agg({
    'Div_UserName' : 'first',
    'Pending_Count' : 'sum',
    'collection_count' : 'sum',
    'proposal_count' : 'sum',
    'retention_count' : 'sum',
    'wrong_number_count' : 'sum',
}).reset_index().rename(columns={'Div_UserID' : 'UserID', 'Div_UserName' : 'UserName' })

In [ ]:
userid_tuple = tuple(divgroup_df['UserID'].unique().tolist())
sql_query = f""" SELECT CallingNumber,UserID FROM Sonata_Connect.dbo.accounts_calling_number_list
    WHERE UserID in {userid_tuple} """
contact_df = pd.read_sql_query(sql_query,engine)
divgroup_df = pd.merge(divgroup_df,contact_df, on='UserID' , how='left').dropna()
divgroup_df

,UserID,UserName,Pending_Count,collection_count,proposal_count,retention_count,wrong_number_count,CallingNumber
0,7535,RAKESH KUMAR CHAUDHARY,190,177,0,9,4,8400333770
2,7662,Sandeep Singh,99,81,0,16,2,8400333741
3,9739,LAKSHMAN SINGH,94,70,0,22,2,7542026601
4,10846,SHASHI BHUSHAN SINGH,195,174,0,17,4,8400333746
5,10849,KESHRAJ YADAV,228,192,0,27,9,8400333551
6,10851,Vinod singh,37,37,0,0,0,8400333705
7,15111,Ajay Kumar Singh,191,120,0,0,71,8400333620
8,15190,JITENDRA YADAV,164,136,0,21,7,8400333680
9,17756,Alok Singh,86,61,0,21,4,7518504947
10,22544,Shiv Kumar Singh,2,0,0,0,2,7518504965


In [ ]:
all_df = pd.concat([brogroup_df ,bmgroup_df, hubgroup_df, regiongroup_df, divgroup_df], axis=0).reset_index(drop=True)

,UserID,UserName,Pending_Count,collection_count,proposal_count,retention_count,wrong_number_count,CallingNumber
0,1709,Ballu,1,1,0,0,0,8960154523
1,1934,Vijay Kumar,2,1,0,1,0,7897997362
2,2114,Vinod Kumar,2,1,0,1,0,7897997269
3,2911,Deepak Singh,1,0,0,1,0,8218048477
4,2949,Dhanpat Singh,2,2,0,0,0,7080200380
...,...,...,...,...,...,...,...,...
999,10851,Vinod singh,37,37,0,0,0,8400333705
1000,15111,Ajay Kumar Singh,191,120,0,0,71,8400333620
1001,15190,JITENDRA YADAV,164,136,0,21,7,8400333680
1002,17756,Alok Singh,86,61,0,21,4,7518504947


In [ ]:
all_contact_sql_query = f""" SELECT CallingNumber,UserID FROM Sonata_Connect.dbo.accounts_calling_number_list """
all_contact_df = pd.read_sql_query(all_contact_sql_query,engine)
all_contact_df

NameError: name 'pd' is not defined